In [1]:
import geopandas as gpd
import requests
import pandas as pd

bounding_box = {
    "xmin": 150.520928,  # Minimum longitude
    "ymin": -34.118347,  # Minimum latitude
    "xmax": 151.343021,  # Maximum longitude
    "ymax": -33.578141   # Maximum latitude
}

for k in [8, 12]:
    # Base URL for querying data
    base_url = f"https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer/{k}/query"
    
    # Function to fetch data incrementally
    def fetch_data_in_chunks(base_url):
        # Parameters for the initial query
        params = {
            "where": "1=1",  # Fetch all records
            "geometry": f"{bounding_box['xmin']},{bounding_box['ymin']},{bounding_box['xmax']},{bounding_box['ymax']}",
            "geometryType": "esriGeometryEnvelope",
            'inSR': '4326',  # Spatial reference of the input geometry (WGS84)
            'spatialRel': 'esriSpatialRelIntersects',  # Spatial relationship
            'outFields': '*',  # Output fields
            'outSR': '4326',  # Spatial reference of the output
            "f": "geojson",  # Get data in GeoJSON format
            "resultOffset": 0,  # Start from the beginning
            "resultRecordCount": 50  # Fetch 1000 records at a time
        }
        all_data = []
        while True:
            response = requests.get(base_url, params=params)
            data = response.json()
            if "features" in data:
                all_data.extend(data["features"])
                if len(data["features"]) < params["resultRecordCount"]:
                    break
                params["resultOffset"] += params["resultRecordCount"]
            else:
                break
        return all_data
    
    # Fetch the data
    data_chunks = fetch_data_in_chunks(base_url)
    
    # Convert to GeoDataFrame
    geospatial_gdf = gpd.GeoDataFrame.from_features(data_chunks).set_crs("EPSG:4326")
    geospatial_gdf.to_feather(f"geo-out/land-parcels-{k}.feather")

KeyboardInterrupt: 

In [ ]:
geospatial_gdf.plot()